In [23]:
import pandas as pd

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss
from math import sqrt

import xgboost
from xgboost import XGBClassifier

from hyperopt import fmin, hp, tpe, space_eval

import gc
import time

In [6]:
train = pd.read_csv('../dataset/train.csv', index_col = 0)
test = pd.read_csv('../dataset/test.csv', index_col = 0)
sample_submission = pd.read_csv('../dataset/sample_submission.csv', index_col = 0)

In [7]:
column_number = {}

for i, column in enumerate(sample_submission.columns):
    column_number[column] = i

def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [8]:
train_x = train.drop(columns = ['type', 'type_num'], axis = 1)
train_y = train['type_num']

test_x = test

In [13]:
dummy_variable = pd.get_dummies(pd.Series(train_x['fiberID']), prefix = 'fibierID')

In [19]:
train_concat_x = pd.concat([train_x, dummy_variable], axis = 1)
train_concat_x.drop(columns = 'fiberID', inplace = True)

In [20]:
train_concat_x.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,...,fibierID_991,fibierID_992,fibierID_993,fibierID_994,fibierID_995,fibierID_996,fibierID_997,fibierID_998,fibierID_999,fibierID_1000
id,,,,,,,,,,,,,,,,,,,,,
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,...,0,0,0,0,0,0,0,0,0,0
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,...,0,0,0,0,0,0,0,0,0,0
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,...,0,0,0,0,0,0,0,0,0,0
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,...,0,0,0,0,0,0,0,0,0,0
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,...,0,0,0,0,0,0,0,0,0,0


In [36]:
def objective(params):
    time1 = time.time()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma' : "{:.3f}".format(params['gamma']),
        'subsample' : "{:.2f}".format(params['subsample']),
        'reg_alpha' : "{:.3f}".format(params['reg_alpha']),
        'learning_rate' : "{:.3f}".format(params['learning_rate']),
        'num_leaves' : "{:.3f}".format(params['num_leaves']),
        'colsample_bytree' : "{:.3f}".format(params['colsample_bytree']),
        'min_child_samples' : "{:.3f}".format(params['min_child_samples']),
        'feature_fraction' : "{:.3f}".format(params['feature_fraction']),
        'bagging_fraction' : "{:.3f}".format(params['bagging_fraction'])
    }
    
    print("****** New Run *******")
    print(f"params = {params}")
    FOLDS = 5
    count = 1
    
    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = 42)
    score_mean = 0
    
    for tr_idx, val_idx in skf.split(train_concat_x, train_y):
        xgb_clf = XGBClassifier(random_state = 42, verbos = False, **params, n_jobs = 4)
        
        X_tr, X_vl = train_concat_x.iloc[tr_idx, :], train_concat_x.iloc[val_idx, :]
        y_tr, y_vl = train_y.iloc[tr_idx], train_y.iloc[val_idx]
        
        xgb_clf.fit(X_tr, y_tr)
        
        score = make_scorer(log_loss)(xgb_clf, X_vl, y_vl)
        
        score_mean += score
        print(f'{count} CV - Score : {round(score,4)}')
        count += 1
        
        time2 = time.time() - time1
        print(f"Total Time Run : {round(time2/60,2)}")
        gc.collect()
        
        print(f"Mean log loss : {score_mean/FOLDS}")
        
        del X_tr, X_vl, y_tr, y_vl, rf_reg, score
        return (score_mean / FOLDS)

In [37]:

space = {
    # 트리의 최대 깊이, GBM과 같습니다.
    # 과대적합을 제어하기 위해 사용, 높은 깊이는 과대적합될 수 있습니다.
    # 특정 샘플에 대한 관계를 학습시킵니다.
    # Cross Validation을 사용하여 튜닝해야 합니다.
    # 일반적인 값: 3 - 10
    'max_depth' : hp.quniform('max_depth', 7, 23, 1),
    
    # reg_alpha: L1 규제 파라미터. L1 규제는 희소성을 상승시킵니다.
    # (meaning pulling weights to 0). objective가 Logistic Regression일 때, 
    # Feature Selection에 유용할 수 있습니다.
    'reg_alpha' : hp.uniform('reg_alpha', 0.01, 0.4),
    
    # reg_lambda: L2 규제 파라미터. 더 작은 가중치를 부여하길 권장합니다.
    # 이 접근 방식은 Zeroing Feature(?)가 의미가 없을 수 있는 트리 모델에서 더 유용할 수 있습니다.
    'reg_lambda': hp.uniform('reg_lambda', 0.01, .4),
    
    # eta: GBM의 학습률과 유사하게 각 단계의 가중치를 줄임으로써 모델을 더욱 견고하게 합니다.
    # 일반적인 값: 0.01 - 0.2
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    
    # colsample_bytree: GBM에서 max_feature와 비슷합니다. 
    # 각 트리에 대해 무작위로 추출할 Feature의 비율입니다.
    # 일반적인 값: 0.5 - 1
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, .9),
    
    # 노드는 분할 결과가 손실 함수를 양수로 감소시킬 때만 분할됩니다. 
    # gamma: 분할에 필요한 최소 loss 감소를 명시합니다. 또한 알고리즘을 보수적으로 만듭니다.
    # 이 값은 손실 함수에 따라 달라질 수 있으므로 조정이 필요합니다.
    'gamma' : hp.uniform('gamma', 0.01, .7),
    
    # 정확도를 더 올려주지만, 모델이 과적합될 수 있습니다.
    # num_leaves: 사용할 리프 노드 수.
    # leaf가 많으면 정확도가 향상되지만 과적합도 발생합니다.
    'num_leaves' : hp.choice('num_leaves', list(range(20,250,10))),
    
    # 리프 노드 당 최소 표본의 수를 지정합니다.
    # leaf으로 그룹화할 최소 표본의 수
    # 이 매개변수는 과대적합에 크게 도움 될 수 있습니다. leaf당 표본의 크기가 클수록 
    # 과적합에서 벗어날 수 있습니다. 반대로 생각하면, 오히려 과소적합으로 이어질 수 있습니다.
    'min_child_samples' : hp.choice('min_child_samples', list(range(100, 250, 10))),
    
    # subsample: 각 하위 트리를 생성할 때, 고려해야 할 관측값의 일부를 나타냅니다.
    'subsample' : hp.choice('subsample', [.2,.4,.5,.6,.7,.8,.9]),
    
    # Feature의 일부를 무작위로 선택합니다.
    # feature_fraction: 훈련에 사용되는 Feature의 subsampling을 제어합니다.
    # 배깅의 경우 실제 훈련 데이터에서 subsampling하는 것과는 대조적입니다.
    # 작을수록 과적합이 줄어듭니다.
    'feature_fraction' : hp.uniform('feature_fraction',.4, .8),
    
    # 무작위로 훈련 데이터를 넣거나, 샘플링합니다.
    'bagging_fraction' : hp.uniform('bagging_fraction', .4, .9),
    
    # bagging_fraction & bagging_freq: 훈련 데이터의 bagging (subsampling)을 활성화합니다.
    # 두 값 모두 배깅을 사용하려면 설정해야 합니다.
    # Frequency는 배깅 사용 빈도를 제어합니다.
    # 더 작은 Fraction과 Frequency는 과적합을 방지합니다.
    
}

In [38]:
# 알고리즘 파라미터 설정
best = fmin(fn = objective,
           space = space,
           algo = tpe.suggest,
           max_evals = 10)

# 최적의 파라미터를 출력
best_params = space_eval(space, best)

****** New Run *******                                                                                                 
params = {'max_depth': 11, 'gamma': '0.079', 'subsample': '0.20', 'reg_alpha': '0.334', 'learning_rate': '0.134', 'num_leaves': '130.000', 'colsample_bytree': '0.613', 'min_child_samples': '220.000', 'feature_fraction': '0.697', 'bagging_fraction': '0.822'}
  0%|                                                                           | 0/10 [34:41<?, ?trial/s, best loss=?]


KeyboardInterrupt: 